# Cookbook: TreeMix
The program TreeMix by Pickrell & Pritchard (2012) is used to infer population splits and admixture from allele frequency data. From the TreeMix documentation: "In the underlying model, the modern-day populations in a species are related to a common ancestor via a graph of ancestral populations. We use the allele frequencies in the modern populations to infer the structure of this graph." <br>https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1002967  <br>https://bitbucket.org/nygcresearch/treemix/wiki/Home  <br>http://genomesunzipped.org/2012/03/identifying-targets-of-natural-selection-in-human-and-dog-evolution.php <br>http://precedings.nature.com/documents/6956/version/1/files/npre20126956-1.pdf

Required Software

In [2]:
## conda install -c ipyrad ipyrad
## conda install -c ipyrad treemix
## conda install -c eaton-lab toytree

In [3]:
import ipyrad.analysis as ipa
import toytree
import toyplot
import numpy as np

In [6]:
print 'ipyrad', ipa.__version__
print 'toytree', toytree.__version__
print 'toyplot', toyplot.__version__
! treemix --version | grep 'TreeMix v. '

ipyrad 0.7.28
toytree 0.1.6
toyplot 0.16.1
TreeMix v. 1.13


### Define the populations
These two dictionaries will be used to parse out SNPs from the ipyrad SNPs output file (*.snps.phy), to filter the SNPs for inclusion in the data set based on missing data (minmap), and to group them into populations (imap).

In [100]:
## a dictionary mapping sample names to 'locality' names
imap = {
#    "La_Seda": ["CC1", "CC2","CC3","CC4"],
#    "8W": ["CC5", "CC6","CC7","CC8","CC9"],
#    "Danis_Creek": ["CC10", "CC11","CC12"],
#    "4W": ["CC13"],
#    "16E": ["CC14","CC15","CC16","CC17"],
#    "Plantation_Road": ["CC18","CC19","CC20","CC21","CC22"],
#    "Gamboa_Forest": ["CC23", "CC24"],
#    }
"La_Seda": ["CC1", "CC2","CC3","CC4"], 
"Bird_Plot": ["CC5", "CC6","CC7","CC8","CC9","CC13","CC14","CC15","CC16","CC17"],
   "Danis_Creek": ["CC10", "CC11","CC12"],
   "Plantation_Road": ["CC18","CC19","CC20","CC21","CC22"],
   "Gamboa_Forest": ["CC23", "CC24"],
    }

## optional: loci will be filtered if they do not have data for at least N samples in each locality. Minimums cannot be <1.
## Nloci were selected arbitrarily
minmap = { 
    "La_Seda": 2,
    "Bird_Plot": 5,
    "Danis_Creek": 1,
    "Plantation_Road": 2,
    "Gamboa_Forest": 1,
    }

In [101]:
ls

analysis-treemix/  both.nex        both.str         ipyrad_log.txt
both.geno          both.phy        both.u.geno      Tree_Mix-trachy.ipynb
both.gphocs        both.snps.map   both.u.snps.phy  untitled.txt
both.hdf5          both.snps.phy   both.ustr
both.loci          both_stats.txt  both.vcf


### Create a Tremix Object
the format for this object is similar to the other ipyrad.analysis objects (see documentation: ????)

In [102]:
t = ipa.treemix(
    name="test",
    data="/home/cardenas.61/output/cluster_analysis/both_outfiles/both.snps.phy",
    imap=imap,
    minmap=minmap,)

In [103]:
#look into the parameter arguments for tree mix!
## you can set additional parameter args here
t.params.m = 1
t.params.root = "Plantation_Road"
t.params.global_ = 1
#t.params.se = 1
t.params

binary      treemix             
bootstrap   0                   
climb       0                   
cormig      0                   
g           (None, None)        
global_     1                   
k           0                   
m           1                   
noss        0                   
root        Plantation_Road     
se          0                   
seed        609271926           

### generate treemix input file


In [104]:
## write treemix input files so you can call treemix from the command line
s = t.write_output_file()

ntaxa 24; nSNPs total 14395; nSNPs written 3453


## The command string
This shows the command string that corresponds to the parameter settings in the Treemix object. You can see that the input file (-i) is the string we enetered in the data field above, and the output prefix (-o) corresponds to the default working directory and the name field that we provided above. In addition, the argument (-m 1) is added because we added that to the params dictionary.

In [105]:
## the command string
print t.command

treemix -i /home/cardenas.61/output/cluster_analysis/both_outfiles/analysis-treemix/test.treemix.in.gz -o /home/cardenas.61/output/cluster_analysis/both_outfiles/analysis-treemix/test -root Plantation_Road -m 1 -seed 609271926 -global


In [106]:
## you can run the command in a notebook by using bash one-liners (!)
! $t.command > analysis-treemix/treemix.log

# Run treemix jobs
Alternatively, you can use the .run() command of the treemix object to run treemix. This is more convenient because the results will automatically be parsed by the treemix object so that they are easily accessible for downstream plotting. In the loop below we run treemix over a range of migration parameters (-m) and with 4 replicates per setting.

In [107]:
## a dictionary for storing treemix objects
tdict = {}

## iterate over values of m
for rep in xrange(4):
    for mig in xrange(4):
        
        ## create new treemix object copy
        name = "mig-{}-rep-{}".format(mig, rep)
        tmp = t.copy(name)

        ## set params on new object
        tmp.params.m = mig
    
        ## run treemix analysis
        tmp.run()
        
        ## store the treemix object
        tdict[name] = tmp

In [108]:
## choose a treemix object from the above analysis
t = tdict['mig-2-rep-3']

In [109]:
## access output files produced by treemix
t.files

cov        ~/output/cluster_analysis/both_outfiles/analysis-treemix/mig-2-rep-3.cov.gz
covse      ~/output/cluster_analysis/both_outfiles/analysis-treemix/mig-2-rep-3.covse.gz
edges      ~/output/cluster_analysis/both_outfiles/analysis-treemix/mig-2-rep-3.edges.gz
llik       ~/output/cluster_analysis/both_outfiles/analysis-treemix/mig-2-rep-3.llik
modelcov   ~/output/cluster_analysis/both_outfiles/analysis-treemix/mig-2-rep-3.modelcov.gz
treeout    ~/output/cluster_analysis/both_outfiles/analysis-treemix/mig-2-rep-3.treeout.gz
vertices   ~/output/cluster_analysis/both_outfiles/analysis-treemix/mig-2-rep-3.vertices.gz

In [110]:
## access the newick string representation of the tree
t.results.tree

'(Plantation_Road:0.0325648,(((La_Seda:0.00861141,Gamboa_Forest:0.0548913):0.0049276,Bird_Plot:0):0.0158149,Danis_Creek:0.046097):0.0325648);'

In [111]:
## access a list of admixture edges
t.results.admixture

[(0, 0.0548913, 2, 0.0, 0.417629), (3, 0.046097, 1, 0.00861141, 0.199906)]

In [112]:
## access the covariance matrix
t.results.modelcov

array([[ 0.0537476 , -0.00941853, -0.0112462 , -0.0105049 , -0.022578  ],
       [-0.00941853,  0.038642  , -0.0092827 ,  0.00067368, -0.0206145 ],
       [-0.0112462 , -0.0092827 ,  0.0101607 ,  0.00393084,  0.00643739],
       [-0.0105049 ,  0.00067368,  0.00393084,  0.0110053 , -0.0051049 ],
       [-0.022578  , -0.0206145 ,  0.00643739, -0.0051049 ,  0.0418599 ]])

If you want, you can also generate plots in R

## Or, you can generate plots in Python using Toytree
The code to produce tree plots with admixture edges in Toytree (But is still in development).
see resource here https://toytree.readthedocs.io/en/latest/1-ethos.html

In [113]:
def _get_admix_point(tre, idx, dist):
    ## parent coordinates
    px, py = tre.verts[idx]
    ## child coordinates
    cx, cy = tre.verts[tre.tree.search_nodes(idx=idx)[0].up.idx]
    ## angle of hypotenuse
    theta = np.arctan((px-cx) / (py-cy))
    ## new coords along the hypot angle
    horz = np.sin(theta) * dist
    vert = np.cos(theta) * dist
    
    ## change x
    a = tre.verts[idx, 0]
    b = tre.verts[idx, 1] 
    a -= abs(horz)
    if py < cy:
        b += abs(vert)
    else:
        b -= abs(vert)
    return a, b

In [114]:
def treemix_plot(tmp, axes):
        
    ## create a toytree object from the treemix tree result
    tre = toytree.tree(newick=tmp.results.tree)
    tre.draw(
        axes=axes,
        use_edge_lengths=True,
        tree_style='c',
        tip_labels_align=True,
        edge_align_style={"stroke-width": 1},
        orient='right'
    );

    ## get coords 
    for admix in tmp.results.admixture:
        ## parse admix event
        pidx, pdist, cidx, cdist, weight = admix
        a = _get_admix_point(tre, pidx, pdist)
        b = _get_admix_point(tre, cidx, cdist)

        ## add line for admixture edge
        mark = axes.plot(
            a = (a[0], b[0]),
            b = (a[1], b[1]),
            style={"stroke-width": 10*weight,
                   "stroke-opacity": 0.95, 
                   "stroke-linecap": "round"}
        )

        ## add points at admixture sink
        axes.scatterplot(
            a = (b[0]),
            b = (b[1]),
            size=8,
            title="weight: {}".format(weight),
        )

    ## add scale bar for edge lengths
    axes.y.show=False
    axes.x.ticks.show=True
    axes.x.label.text = "Drift parameter"
    return axes

### Draw a single result


In [115]:
toyplot.__version__

'0.16.1'

In [116]:
## select a result
tmp = tdict["mig-1-rep-0"]

## draw the tree similar to the Treemix plotting R code
canvas = toyplot.Canvas(width=450, height=450)
axes = canvas.cartesian(padding=25, margin=75)
axes = treemix_plot(tmp,axes,)

Bird_Plot Gamboa_Forest La_Seda Danis_Creek Plantation_Road weight: 0.408644 -0.10 -0.05 0.00 0.05 Drift parameter

## Draw replicate runs

In [117]:
canvas = toyplot.Canvas(width=1000, height=1400)
idx = 0
for mig in range(4):
    for rep in range(4):
        tmp = tdict["mig-{}-rep-{}".format(mig, rep)]
        ax = canvas.cartesian(grid=(4, 4, idx), padding=25, margin=(25, 50, 100, 25))
        ax = treemix_plot(tmp, ax)
        idx += 1

Bird_Plot Gamboa_Forest La_Seda Danis_Creek Plantation_Road -0.10 -0.05 0.00 0.05 Drift parameter Bird_Plot Gamboa_Forest La_Seda Danis_Creek Plantation_Road -0.10 -0.05 0.00 0.05 Drift parameter Bird_Plot Gamboa_Forest La_Seda Danis_Creek Plantation_Road -0.10 -0.05 0.00 0.05 Drift parameter Bird_Plot Gamboa_Forest La_Seda Danis_Creek Plantation_Road -0.10 -0.05 0.00 0.05 Drift parameter Bird_Plot Gamboa_Forest La_Seda Danis_Creek Plantation_Road weight: 0.408644 -0.10 -0.05 0.00 0.05 Drift parameter Bird_Plot Gamboa_Forest La_Seda Danis_Creek Plantation_Road weight: 0.408644 -0.10 -0.05 0.00 0.05 Drift parameter Bird_Plot Gamboa_Forest La_Seda Danis_Creek Plantation_Road weight: 0.408644 -0.10 -0.05 0.00 0.05 Drift parameter Bird_Plot Gamboa_Forest La_Seda Danis_Creek Plantation_Road weight: 0.408644 -0.10 -0.05 0.00 0.05 Drift parameter Gamboa_Forest La_Seda Bird_Plot Danis_Creek Plantation_Road weight: 0.417629 weight: 0.199906 -0.10 -0.05 0.00 0.05 Drift parameter Gamboa_Forest La_Seda Bird_Plot Danis_Creek Plantation_Road weight: 0.417629 weight: 0.199906 -0.10 -0.05 0.00 0.05 Drift parameter Gamboa_Forest La_Seda Bird_Plot Danis_Creek Plantation_Road weight: 0.417629 weight: 0.199906 -0.10 -0.05 0.00 0.05 Drift parameter Gamboa_Forest La_Seda Bird_Plot Danis_Creek Plantation_Road weight: 0.417629 weight: 0.199906 -0.10 -0.05 0.00 0.05 Drift parameter Gamboa_Forest La_Seda Bird_Plot Danis_Creek Plantation_Road weight: 0.417616 weight: 0.199894 weight: 0.179949 -0.15 -0.10 -0.05 0.00 0.05 0.10 Drift parameter Gamboa_Forest La_Seda Bird_Plot Danis_Creek Plantation_Road weight: 0.417616 weight: 0.199894 weight: 0.179949 -0.15 -0.10 -0.05 0.00 0.05 0.10 Drift parameter Gamboa_Forest La_Seda Bird_Plot Danis_Creek Plantation_Road weight: 0.417616 weight: 0.199894 weight: 0.179949 -0.15 -0.10 -0.05 0.00 0.05 0.10 Drift parameter Gamboa_Forest La_Seda Bird_Plot Danis_Creek Plantation_Road weight: 0.417616 weight: 0.199894 weight: 0.179949 -0.15 -0.10 -0.05 0.00 0.05 0.10 Drift parameter

## Graph the correlation plot
still needs a little work on the X axis... but you get the picture.

In [118]:
## grab names from the tree
tre = toytree.tree(tmp.results.tree)

lnames = toyplot.locator.Explicit(
    locations=range(len(tre.get_tip_labels())),
    labels=tre.get_tip_labels(),
)

## get a colormap and plot the matrix
cmap = toyplot.color.diverging.map("BlueRed", domain_min=-0.05, domain_max=0.05)
canvas, table = toyplot.matrix(
              (tmp.results.cov, cmap),
               width=600, 
               height=600, 
               bshow=True,
               tshow=False,
               llocator=lnames,
               blocator=lnames,
               );

## add a color scale
tlocs = np.linspace(tmp.results.cov.min(), tmp.results.cov.max(), 5)
tlabs = ["{:.2f}".format(i) for i in tlocs]
canvas.color_scale(cmap, 
                   x1=100, x2=500, y1=50, y2=50, 
                   ticklocator=toyplot.locator.Explicit(
                       locations=tlocs, 
                       labels=tlabs,
                   ));

Plantation_Road 0.0537505 -0.00933945 -0.0126695 -0.0109862 -0.0207553 Danis_Creek -0.00933945 0.0385246 -0.00787316 0.00113688 -0.0224488 Bird_Plot -0.0126695 -0.00787316 0.010154 0.0039443 0.00644445 La_Seda -0.0109862 0.00113688 0.0039443 0.0110082 -0.00510314 Gamboa_Forest -0.0207553 -0.0224488 0.00644445 -0.00510314 0.0418628 Plantation_Road Danis_Creek Bird_Plot La_Seda Gamboa_Forest -0.02 -0.00 0.02 0.03 0.05

# read the literature on tree mix!

~1~ http://precedings.nature.com/documents/6956/version/1/files/npre20126956-1.pdf

~2~ 
The tree predicts a variance/covariance matrix of allele frequencies (this is W in the notation of ~1~). For any given SNP, I compute the sample variance/covariance matrix (let’s call this V), and then compute the sum of squared differences between the entries of V and W. I then find the scaling factor that minimizes this sum of squares; i.e., I find the scalar x that minimizes the sum of squared differences between the entries of V and xW. The remaining sum of squared differences is a measure of the “badness of fit” of the SNP to the tree. Obviously there are a number of complications to the interpretation of this number (e.g., it will be larger for SNPs with a larger x, and I make no attempt at accounting for the correlation between different entries of the matrix).

~3~
The “interesting” SNPs will be those with the worst fit to the tree. You should show the most “interesting” SNPs from our data; report their chromosomal position, the nearest gene, and the phenotype influenced by variation in this region (if one is known; do literature search!)